In [ ]:
import pathlib
import pandas as pd
from tqdm import tqdm
import nibabel as nib

from BodyCompositionCalcV2 import calculate_body_composition_measures, calculate_body_composition_volume_measures

In [ ]:
data_path = pathlib.Path('./training-data')
output_folder = pathlib.Path('./output')

In [ ]:
clinical_df = pd.read_csv('./training-data/clinical.csv', index_col = 0)
clinical_df = clinical_df.set_index('PatientID')

In [4]:
mean_height = clinical_df['Height_in_meters'].mean()
clinical_df['Height_in_meters'] = clinical_df['Height_in_meters'].fillna(mean_height)

In [ ]:
l3_output_name = 'body_composition_output_l3_v1.csv'
t12_output_name = 'body_composition_output_t12_v1.csv'
volumetric_output_name = 'body_composition_output_volumetric_v1.csv'

In [ ]:
l3_list = []
t12_list = []
volume_list = []

i = 0
niftis = data_path.glob('*/*.nii.gz')

for nifti_path in tqdm(niftis, total=len(niftis), desc='Calculating Body Composition Metrics', unit='CTs'):
    person_id = nifti_path.parent.name
    try:
        height = clinical_df.loc[person_id].Height_in_meters
    except:
        height = mean_height
    
    tqdm.write(f"Processing Patient ID: {person_id}")

    # load images
    base_dir = nifti_path.parent
    nifti_image = nib.load(nifti_path)
    masks = {
        'total_img' : nib.load(base_dir / f"{person_id}_total_masks.nii"),
        'body_img' : nib.load(base_dir / f"{person_id}_body_masks.nii"),
        'tissue_img' : nib.load(base_dir / f"{person_id}_tissue_masks.nii")
    }
    
    try:
        body_comp = calculate_body_composition_measures(
            nifti_image,
            masks,
            height = height,
            fat_hu_range = (-190, -30),
            calc_threshold = 'adaptive',
            vertebra = 'L3',
            output_unit = 'cm'
        )
        body_comp['PersonId'] = person_id
        l3_list.append(body_comp)
    except Exception as e: 
        tqdm.write(f"error L3 error: {e}")

    try:
        body_comp = calculate_body_composition_measures(
            nifti_image,
            masks,
            height = height,
            fat_hu_range = (-190, -30),
            calc_threshold = 'adaptive',
            vertebra = 'T12',
            output_unit = 'cm'
        )
        body_comp['PersonId'] = person_id
        t12_list.append(body_comp)
    except Exception as e: 
        tqdm.write(f"error T12 error: {e}")

    try:
        body_comp = calculate_body_composition_volume_measures(
            nifti_image,
            masks,
            height = height,
            fat_hu_range = (-190, -30),
            calc_threshold = 'adaptive',
            output_unit = 'cm'
        )
        body_comp['PersonId'] = person_id
        volume_list.append(body_comp)
    except Exception as e: 
        tqdm.write(f"error volume error: {e}")

    if i % 10 == 0:
        # dump intermediate state
        pd.DataFrame(t12_list).to_csv(output_folder /  t12_output_name)
        pd.DataFrame(l3_list).to_csv(output_folder /  l3_output_name)
        pd.DataFrame(volume_list).to_csv(output_folder /  volumetric_output_name)
    i += 1

In [ ]:
l3_df = pd.DataFrame(l3_list)
l3_df.to_csv(output_folder / l3_output_name)
l3_df

In [ ]:
t12_df = pd.DataFrame(t12_list)
t12_df.to_csv(output_folder / t12_output_name)
t12_df

In [ ]:
volume_df = pd.DataFrame(volume_list)
volume_df.to_csv(output_folder / volumetric_output_name)
volume_df

In [ ]:
print("fin.")